In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Desktop/Covered_regions.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/Madhu/KHCDPRGPTTL13/KHCDPRGPTTL13_annotated_indel.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL13_annotated_indel.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

    
data = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHAIGHGPTTL207_annotated_indel.vcf', comment='#', sep= '\t', header=None, low_memory=False)
data.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = data['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
data = pd.concat([data, sample_cols], axis=1)
data = data[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]

# Create empty columns
columns = ['ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID', 'SLO',
           'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV', 'TPA', 'CFL', 'GNO',
           'VLD', 'ASP', 'ASS', 'Ref', 'U3', 'U5', 'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC',
           'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO',
           'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM', 'ClinVar',
           'ClinVar_CLNSIG']

for col in columns:
    data[col] = ''

# Populate columns based on 'INFO' values
for i, row in data.iterrows():
    info = row['INFO']
    items = info.split(';')
    for item in items:
        key_value = item.split('=')
        key = key_value[0]
        if key in columns:
            if len(key_value) > 1:
                value = key_value[1]
                data.at[i, key] = f"{key}={value}"
            else:
                data.at[i, key] = key
        else:
            data.at[i, key] = 'null'


            
data["Gene_Name"] = data["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
data['Gene Name'] = data['Gene_Name'].apply(lambda x: ','.join([segment.split(':')[0] for segment in x.split('|')]) if pd.notnull(x) else '')


data = data[['CHROM', 'POS', 'Gene Name', 'rsID', 'REF', 'ALT', 'GT',
       'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF',
       'ADR', 'ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD',
       'dbSNPBuildID', 'SLO', 'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON',
       'RS', 'RV', 'TPA', 'CFL', 'GNO', 'VLD', 'ASP', 'ASS', 'Ref', 'U3', 'U5',
       'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC', 'DSS', 'SYN', 'KGPhase3', 'CAF',
       'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO', 'INT', 'G5',
       'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM', 'ClinVar', 'ClinVar_CLNSIG']]

data['rsid'] = data['rsID'].str.split(';')
data = data.explode('rsid')

df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHAIGHGPTTL207_indel_clinical_significance.xlsx')
df = df.rename(columns={'gene_name': 'Gene name', 'zygocity':'zygosity'})

merged = pd.merge(data, df, on = 'rsid', how = 'outer', sort=False)

merged['Gene Name'] = merged['Gene Name'].fillna(merged['Gene name'])

cond_genes = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Park_cere_genes.xlsx')

merged['Gene Match'] = 'No'
merged['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in cond_genes['Gene Name'].values:
                merged.at[index, 'Gene Match'] = 'Yes'
                merged.at[index, 'Matched_Gene'] = gene
                break
    
cond_genes = cond_genes.rename({'Gene Name':'Matched_Gene', 'Gene_Match':'Gene Match'}, axis=1)
cond_genes

merged = merged.drop(columns=['Gene Match'], axis=1)
merged_2 = pd.merge(merged, cond_genes, on= 'Matched_Gene', how = 'left', sort = False)
merged_2['Gene Match'] = merged_2['Gene Match'].fillna('No')
merged_2

merged_2['Consequence'] = merged_2['consequence'].astype(str).apply(lambda x: x.replace('&', ',').replace('_', ' ').replace("'", '').replace("-", ' '))
merged_2['Consequence'] = merged_2['Consequence'].str.split(',').str[0]

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1 = df_1.rename({'consequence':'Consequence'}, axis =1)


merged_1 = pd.merge(merged_2, df_1, on='Consequence', how='left', sort=False)


merged_1 = merged_1[['CHROM', 'POS', 'Gene Name', 'Gene Match', 'rsID', 'REF', 'ALT', 'GT', 'GQ', 'SDP',
       'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'ADP',
       'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID',
       'SLO', 'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV',
       'TPA', 'CFL', 'GNO', 'VLD', 'ASP', 'ASS', 'Ref', 'U3', 'U5', 'TOPMED',
       'WGT', 'MTP', 'LSD', 'NOC', 'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC',
       'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO', 'INT', 'G5', 'OM', 'PMC', 'SSR',
       'RSPOS', 'HD', 'PM', 'ClinVar', 'ClinVar_CLNSIG', 'rsid', 'allele',
       'zygosity', 'consequence', 'Consequence_score', 'clinical_significance',
       'associated_diseases', 'review_status', 'origin', 'variant_type',
       'variant_subtype', 'Phargkb_ann_exists', 'is_mutation',
       'Variant_is_precious']]\

merged_1 = merged_1.fillna('NA')
merged_1

#merged_1.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/InDel_files/KHAIGHGPTTL207_InDel_final.xlsx', index=False)

,CHROM,POS,Gene Name,Gene Match,rsID,REF,ALT,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,ADP,WT,HET,HOM,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,Ref,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,ClinVar,ClinVar_CLNSIG,rsid,allele,zygosity,consequence,Consequence_score,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious
0,chr1,1299382.0,ACAP3,No,rs143128930,AG,A,0/1,97,50,50,24,26,52%,1.7375E-10,18,6,17,9,ADP=50,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=134,,,,R5,,,,COMMON=1,RS=143128930,,,,,VLD,ASP,,,,,"TOPMED=0.91126720183486238,0.08873279816513761",WGT=1,,,,,,KGPhase3,"CAF=0.8806,0.1194",VC=DIV,,KGPhase1,,VP=0x0500000a000515003e000200,SAO=0,INT,G5,,,SSR=0,RSPOS=1299383,,,,,rs143128930,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,chr1,1331945.0,TAS1R3,No,rs200330269,G,GC,0/1,48,41,41,26,14,34.15%,1.5388E-5,12,14,2,12,ADP=41,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=1,RS=200330269,,,,,VLD,ASP,,,,,"TOPMED=0.82975758154943934,0.17023445463812436...",WGT=1,,,,,,KGPhase3,"CAF=0.8105,0.1895,.",VC=DIV,,KGPhase1,,VP=0x05000008000515003e000200,SAO=0,INT,G5,,,SSR=0,RSPOS=1331945,,,,,rs200330269,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,chr1,1353987.0,MXRA8,No,rs140777846,CTG,C,0/1,219,70,70,19,51,72.86%,1.2075E-22,12,7,34,17,ADP=70,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=134,SLO,,,,,,G5A,COMMON=1,RS=140777846,,,,GNO,VLD,ASP,,,,,"TOPMED=0.13763857033639143,0.86236142966360856",WGT=1,,,,,,KGPhase3,"CAF=0.1793,0.8207",VC=DIV,,,,VP=0x050100080005170126000200,SAO=0,INT,G5,,,SSR=0,RSPOS=1353988,,,,,rs140777846,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,chr1,1355779.0,MXRA8,No,rs201260508,GA,G,0/1,88,29,29,8,21,72.41%,1.2841E-9,7,1,8,13,ADP=29,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=0,RS=201260508,,,,,,ASP,,,,,"TOPMED=0.22555109582059123,0.77444890417940876",WGT=1,,,,,,KGPhase3,"CAF=0,1",VC=DIV,,KGPhase1,,VP=0x05000008000500003e000200,SAO=0,INT,,,,SSR=0,RSPOS=1355780,,,,,rs201260508,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,chr1,1657358.0,CDK11B,No,rs377230281,T,TA,0/1,78,112,112,88,24,21.43%,1.4999E-8,65,23,4,20,ADP=112,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=138,,,,,,,,,RS=377230281,,,,,,ASP,,,,,"TOPMED=0.98587219673802242,0.01412780326197757",WGT=1,,,,,,,,VC=DIV,,,,VP=0x050000080005000002000200,SAO=0,INT,,,,SSR=0,RSPOS=1657358,,,,,rs377230281,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,NA,NA,KAT6B,No,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,rs71929101,"['GGAA', G]",Heterozygous,inframe_deletion,6/10,Benign/Likely_benign,Genitopatellar_syndrome|not_specified|not_prov...,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,Yes
4954,NA,NA,KAT6B,No,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,rs71929101,"['GGAA', G]",Heterozygous,inframe_deletion,6/10,Benign,Genitopatellar_syndrome|not_provided,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes
4955,NA,NA,GNPTAB,No,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,rs76300806,"['AGCC', A]